In [2]:
import jax
import numpy as np
from scipy.stats.contingency import association
import pandas as pd
jax.config.update('jax_platform_name', 'cpu')

# import cudf

/home/zihaog/miniconda3/envs/py310/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
# a = np.random.normal(0, 1, size = 5000)
# b = np.random.normal(0, 1, size = 5000)

a = np.random.randint(0, 5, size = 5000)
b = np.random.randint(0, 3, size = 5000)


In [4]:
%%timeit
df_xtab = pd.crosstab(a,b)
result = association(df_xtab, method='cramer')
result

7.32 ms ± 550 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [143]:
%%timeit
# crosstab
xtab = np.unique(np.stack([a,b]), axis = 1, return_counts= True)

index = len(np.unique(xtab[0][0]))
col = len(np.unique(xtab[0][1]))
xt = np.zeros((index,col), dtype=np.int32)
for idx, count in zip(xtab[0].T, xtab[1]):
    xt[idx[0], idx[1]] = count

association(xt, method='cramer')

3.81 ms ± 103 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [144]:
%%timeit
# crosstab
cols = a, b
uniq_vals_all_cols, idx = zip( *(np.unique(col, return_inverse=True) for col in cols) )
shape_xt = [uniq_vals_col.size for uniq_vals_col in uniq_vals_all_cols]
dtype_xt = 'uint'
xt = np.zeros(shape_xt, dtype=dtype_xt)
np.add.at(xt, idx, 1)
uniq_vals_all_cols, xt

association(xt, method='cramer')

In [62]:
import numba

In [138]:
@numba.jit(nopython=True)
def numba_xtab(a,b):
    uniq_vals_all_col_a = np.unique(a)
    uniq_vals_all_col_b = np.unique(b)
    shape_xt = (uniq_vals_all_col_a.size, uniq_vals_all_col_b.size)
    xt = np.zeros(shape_xt, dtype=np.int32)
    for i, j in zip(a,b):
        xt[i,j] += 1
    return xt



In [142]:
%%timeit
xt = numba_xtab(a,b)
association(xt, method='cramer')

590 µs ± 39.1 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
